In [96]:
import urllib.request

from pyspark.sql import SparkSession 
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

file_name = "shakespeare.txt"

urllib.request.urlretrieve(f"http://www.gutenberg.org/files/100/100-0.txt", file_name) 

stop_words_df = (spark.read.text("./stopwords.txt")
                .select(F.lower(F.col("value")).alias("words_lower"))
                .select(F.rtrim(F.col("words_lower")).alias("word")))

word_counts_df = (spark.read.text(f"./{file_name}")
 .select(F.lower(F.col("value")).alias("words_lower"))
 .select(F.split(F.col("words_lower"), " ").alias("words_split"))
 .select(F.explode(F.col("words_split")).alias("word"))
 .select(F.regexp_extract(F.col("word"), "[a-z]+", 0).alias("word"))
 .where(F.ltrim(F.col("word")) != "")
 .join(stop_words_df, "word",  "leftanti")
 .groupBy("word")
 .count()
 .orderBy(F.col("count").desc()))